In [ ]:
import matplotlib.pyplot as plt

In [ ]:
# Units: s/byte
T_copy = 1/(11446.6*1024*1024)

# Function that takes the number of points as argument and returns the time it takes to send AND recv that many
T_MPI_sendrecv = lambda x: max(0.025,0.0000017543754*x-0.1000854)/1000

In [ ]:
# L, R, T, B
halo = [1,1,1,1]

In [ ]:
T = []
for i in range(100,10000,100):
    num_halo = 8 * ( (halo[0]+halo[1])*i + (halo[2]+halo[3])*(i-(halo[0]+halo[1])) )
    T.append(num_halo*T_copy + T_MPI_sendrecv(num_halo) + num_halo*T_copy)

In [ ]:
x_exp = [100, 500, 1000, 2000, 3000, 4000, 5000, 7500, 10000]
T_exp = []

In [ ]:
line1, = plt.plot(range(100,10000,100),T, label="Performance Model")
line2, = plt.plot(x_exp, T_exp, label="Experiments")

plt.legend(handles=[line1, line2], loc=2)

plt.show()